In [5]:
import pymysql

# your username and password go here!
creds = {'user': 'brian', 
         'pswd': 'farley'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [6]:
import pandas as pd

In [7]:
getCoreDates = "SELECT page_id, page_info_pulled, page_touched FROM core"
cur.execute(getCoreDates)
core_dates = pd.DataFrame(list(cur.fetchall()),columns=['page_id','page_info_pulled','page_touched'])

In [8]:
getMentionedDates = "SELECT * from current_events"
cur.execute(getMentionedDates)
mentioned_dates = pd.DataFrame(list(cur.fetchall()),columns=['page_id','medianDate','mostRecentDate'])

In [12]:
dates = pd.merge(core_dates,mentioned_dates,on='page_id')

In [13]:
dates

,page_id,page_info_pulled,page_touched,medianDate,mostRecentDate
0,25,20160709000000,20160705210334,2013-03-22,2013-03-22
1,290,20160709000000,20160709170824,2014-05-28,2015-05-11
2,307,20160709000000,20160709073542,1862-02-06,1999-06-28
3,308,20160709000000,20160708133344,None,None
4,316,20160719000000,20160717042830,None,None
5,572,20160719000000,20160716032534,None,None
6,594,20160709000000,20160704191912,None,None
7,615,20160719000000,20160717004623,None,None
8,620,20160709000000,20160705100358,1945-08-24,2009-07-17
9,621,20160719000000,20160717192413,None,None


In [21]:
import datetime

def WikiDateToDate(date_int):
    date_int = str(date_int)
    year = date_int[:4]
    month = date_int[4:6]
    day = date_int[6:8]
    return datetime.date(int(year),int(month),int(day))

(WikiDateToDate(dates['page_info_pulled'][0]) - WikiDateToDate(dates['page_touched'][0])) / datetime.timedelta(days=1)

4.0

In [25]:
dates['days_since_touch'] =  ((dates['page_info_pulled'].apply(WikiDateToDate) -
                              dates['page_touched'].apply(WikiDateToDate))
                              / datetime.timedelta(days=1))

In [27]:
dates['days_since_touch'] = dates['days_since_touch'].astype(int)

In [33]:
def NullableMedianDiff(x):
    if x['medianDate'] is None:
        return None
    else:
        return ((WikiDateToDate(x['page_info_pulled']) - x['medianDate']) / datetime.timedelta(days=1))
    
def NullableMostRecentDiff(x):
    if x['mostRecentDate'] is None:
        return None
    else:
        return(((WikiDateToDate(x['page_info_pulled'])) - x['mostRecentDate']) / datetime.timedelta(days=1))

In [34]:
dates['days_since_median'] = dates.apply(NullableMedianDiff, axis=1)
dates['days_since_most_recent'] = dates.apply(NullableMostRecentDiff, axis=1)

In [39]:
def MedianWithinMonth(x):
    if x['days_since_median'] is None:
        return 0
    elif x['days_since_median'] > 0 and x['days_since_median'] <= 30:
        return 1
    else:
        return 0
    
def MedianWithinWeek(x):
    if x['days_since_median'] is None:
        return 0
    elif x['days_since_median'] > 0 and x['days_since_median'] <= 7:
        return 1
    else:
        return 0

def MedianInFuture(x):
    if x['days_since_median'] is None:
        return 0
    elif x['days_since_median'] < 0:
        return 1
    else:
        return 0
    
def NoDates(x):
    if x['medianDate'] is None:
        return 1
    else:
        return 0
    
def MostRecentWithinMonth(x):
    if x['days_since_most_recent'] is None:
        return 0
    elif x['days_since_most_recent'] > 0 and x['days_since_most_recent'] <= 30:
        return 1
    else:
        return 0
    
def MostRecentWithinWeek(x):
    if x['days_since_most_recent'] is None:
        return 0
    elif x['days_since_most_recent'] > 0 and x['days_since_most_recent'] <= 7:
        return 1
    else:
        return 0 

def MostRecentInFuture(x):
    if x['days_since_most_recent'] is None:
        return 0
    elif x['days_since_most_recent'] < 0:
        return 1
    else:
        return 0

In [40]:
dates['noDates'] = dates.apply(NoDates,axis=1)
dates['medianWithinMonth'] = dates.apply(MedianWithinMonth,axis=1)
dates['medianWithinWeek'] = dates.apply(MedianWithinWeek,axis=1)
dates['medianInFuture'] = dates.apply(MedianInFuture,axis=1)
dates['mostRecentWithinMonth'] = dates.apply(MostRecentWithinMonth,axis=1)
dates['mostRecentWithinWeek'] = dates.apply(MostRecentWithinWeek,axis=1)
dates['mostRecentInFuture'] = dates.apply(MostRecentInFuture,axis=1)

In [45]:
dates[dates['mostRecentInFuture'] == 1]

,page_id,page_info_pulled,page_touched,medianDate,mostRecentDate,days_since_touch,days_since_median,days_since_most_recent,noDates,medianWithinMonth,medianWithinWeek,medianInFuture,mostRecentWithinMonth,mostRecentWithinWeek,mostRecentInFuture
231,4314,20160709000000,20160707131524,1998-10-20,2017-02-04,2,6472.0,-210.0,0,0,0,0,0,0,1
457,7463,20160709000000,20160705100358,1991-06-30,2016-09-22,4,9141.0,-75.0,0,0,0,0,0,0,1
601,9770,20160719000000,20160717012657,2282-01-16,3340-11-30,2,-96970.0,-483715.0,0,0,0,1,0,0,1
935,14810,20160709000000,20160705203658,2002-03-16,2016-10-02,4,5229.0,-85.0,0,0,0,0,0,0,1
992,15782,20160709000000,20160709063132,1814-11-01,2016-10-24,0,73665.0,-107.0,0,0,0,0,0,0,1
1018,16027,20160709000000,20160705103007,2008-09-04,2016-11-19,4,2865.0,-133.0,0,0,0,0,0,0,1
1074,16768,20160709000000,20160708181856,2003-03-19,2016-08-05,1,4861.0,-27.0,0,0,0,0,0,0,1
1084,16882,20160719000000,20160717103932,1949-07-12,2016-10-31,2,24479.0,-104.0,0,0,0,0,0,0,1
1167,18119,20160709000000,20160710003852,1998-03-12,2016-07-10,-1,6694.0,-1.0,0,0,0,0,0,0,1
1209,18787,20160709000000,20160707235126,2010-11-10,2016-08-20,2,2068.0,-42.0,0,0,0,0,0,0,1


In [46]:
forPush = dates[['page_id','days_since_touch','noDates','medianWithinMonth','medianWithinWeek','medianInFuture',
                'mostRecentWithinMonth','mostRecentWithinWeek','mostRecentInFuture']]

In [48]:
recordsForDTF = forPush.to_dict(orient="records")

In [58]:
def chunk(l,n):
    for i in range(0,len(l),n):
        yield l[i:i+n]

chunks = chunk(recordsForDTF,1000)

In [59]:
import datetime
DTFinsert = "INSERT INTO date_time_features({}) VALUES {}"

for ix, chunk in enumerate(chunks):
    now = datetime.datetime.now().time()
    print("[{:02d}:{:02d}:{:02d}] inserting chunk {}".format(now.hour,now.minute,now.second,ix))
    cols = list(chunk[0].keys())
    valstring = []
    for record in chunk:
        values = [str(x) for x in list(record.values())]
        valstring.append("({})".format(",".join(values)))
    insertme = DTFinsert.format(",".join(cols),",".join(valstring))
    cur.execute(insertme)
    connection.commit()


[14:59:54] inserting chunk 0
[14:59:54] inserting chunk 1
[14:59:54] inserting chunk 2
[14:59:55] inserting chunk 3
[14:59:55] inserting chunk 4
[14:59:55] inserting chunk 5
[14:59:55] inserting chunk 6
[14:59:55] inserting chunk 7
[14:59:55] inserting chunk 8
[14:59:55] inserting chunk 9
[14:59:55] inserting chunk 10
[14:59:55] inserting chunk 11
[14:59:55] inserting chunk 12
[14:59:55] inserting chunk 13
[14:59:55] inserting chunk 14
[14:59:55] inserting chunk 15
[14:59:56] inserting chunk 16
[14:59:56] inserting chunk 17
[14:59:56] inserting chunk 18
[14:59:56] inserting chunk 19
[14:59:56] inserting chunk 20
[14:59:56] inserting chunk 21
[14:59:56] inserting chunk 22
[14:59:56] inserting chunk 23
[14:59:56] inserting chunk 24
[14:59:56] inserting chunk 25
[14:59:56] inserting chunk 26
[14:59:56] inserting chunk 27
[14:59:56] inserting chunk 28
[14:59:56] inserting chunk 29
[14:59:56] inserting chunk 30
[14:59:56] inserting chunk 31
[14:59:56] inserting chunk 32
[14:59:57] inserting

In [60]:
cols = ['page_id','page_title','page_is_ep','noDates','medianWithinMonth','medianWithinWeek','medianInFuture',
       'mostRecentWithinWeek','mostRecentWithinMonth','mostRecentInFuture']
getSomeData = """SELECT {} FROM core INNER JOIN date_time_features USING(page_id) WHERE
                 page_batch != 99"""
Qdata = getSomeData.format(",".join(cols))
cur.execute(Qdata)
dates_df = pd.DataFrame(list(cur.fetchall()),columns=cols)

In [71]:
dataCols = ['noDates','medianWithinMonth','medianWithinWeek','medianInFuture','mostRecentWithinMonth','mostRecentWithinWeek','mostRecentInFuture']
ep_freq = dates_df[dates_df['page_is_ep'] == 1][dataCols].sum(axis=0) 
nep_freq = dates_df[dates_df['page_is_ep'] == 0][dataCols].sum(axis=0) 

In [72]:
ep_freq = ep_freq / len(dates_df[dates_df['page_is_ep'] == 1])
nep_freq = nep_freq / len(dates_df[dates_df['page_is_ep'] == 0])

In [73]:
ep_freq

noDates                  0.302418
medianWithinMonth        0.002657
medianWithinWeek         0.001063
medianInFuture           0.002126
mostRecentWithinMonth    0.038134
mostRecentWithinWeek     0.016078
mostRecentInFuture       0.020330
dtype: float64

In [74]:
nep_freq

noDates                  0.413324
medianWithinMonth        0.003524
medianWithinWeek         0.002217
medianInFuture           0.001161
mostRecentWithinMonth    0.011429
mostRecentWithinWeek     0.004606
mostRecentInFuture       0.006242
dtype: float64